In [1]:
from typing import List
from haystack import component, Document
from newspaper import Article
import requests

In [2]:
@component
class HackernewsNewestFetcher():

  @component.output_types(articles=List[Document])
  def run(self, last_k: int):
    newest_list = requests.get(url='https://hacker-news.firebaseio.com/v0/newstories.json?print=pretty')
    articles = []
    for id in newest_list.json()[0:last_k]:
      article = requests.get(url=f"https://hacker-news.firebaseio.com/v0/item/{id}.json?print=pretty")
      if 'url' in article.json():
        articles.append(article.json()['url'])

    docs = []
    for url in articles:
      try:
        article = Article(url)
        article.download()
        article.parse()
        docs.append(Document(content=article.text, meta={'title': article.title, 'url': url}))
      except:
        print(f"Couldn't download {url}, skipped")
    return {'articles': docs}

In [3]:
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import OpenAIGenerator

In [4]:
import os
os.environ['OPENAI_API_KEY']='sk-111111111111111111111111111111111111111111111111'
os.environ['OPENAI_API_BASE']='http://127.0.0.1:5000/v1'

In [5]:
import requests
model_info_url = 'http://127.0.0.1:5000/v1/internal/model/info'
resp = requests.get(model_info_url)
model = resp.json()['model_name']

print(model)

Mixtral-8x7B-instruct-exl2-6.0bpw


In [6]:
prompt_template = """
You will be provided a few of the latest posts in HakcerNews, followed by their URL.
For each post, provide a brief summary followed by the URL the full post can be found in.

Posts:
{% for article in articles %}
  {{article.content}}
  URL: {{article.meta['url']}}
{% endfor %}
"""

In [7]:
prompt_builder = PromptBuilder(template=prompt_template)

In [8]:
llm = OpenAIGenerator(api_base_url=os.getenv('OPENAI_API_BASE'))

In [9]:
fetcher = HackernewsNewestFetcher()

In [10]:
pipe = Pipeline()
pipe.add_component("hackernews_fetcher", fetcher)
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)

In [11]:
pipe.connect("hackernews_fetcher.articles", "prompt_builder.articles")
pipe.connect("prompt_builder.prompt", "llm.prompt")

In [12]:
result = pipe.run(data={"hackernews_fetcher": {"last_k": 3}})

In [13]:
print(result['llm']['replies'][0])

Summary: The post discusses how rural areas in the US are seeing increased investment in data centers due to the growing demand for infrastructure to support AI applications. The post provides examples of hyperscalers like AWS, Meta, Microsoft, and Google investing in data centers in states like Virginia, Mississippi, Idaho, Texas, and Ohio. The post also mentions how these projects are driving the need for rapid deployment of energy infrastructure.

Source: <https://siteselection.com/issues/2024/mar/how-data-centers-are-reshaping-rural-america.cfm>

Summary: The post explores a scenario where an attacker can plant code to implement a "Fake Lockdown Mode" on a compromised device. The post explains that if a hacker has already infiltrated the device, activating Lockdown Mode will not prevent the execution of malicious payloads. The post then demonstrates how malware could trick the user into thinking Lockdown Mode is active, allowing the malware to continue running in the background.

S